In [1]:
import spacy
from spacy.language import Language
import os
import docx2txt
import pandas as pd
import warnings
warnings.filterwarnings("ignore")
import re

In [2]:
nlp = spacy.load('en_core_web_lg',disable = ['ner', 'parser'])
spacy_stopwords = spacy.lang.en.stop_words.STOP_WORDS
print(nlp.pipe_names)

['tok2vec', 'tagger', 'attribute_ruler', 'lemmatizer']


In [3]:
import nltk
from nltk.corpus import stopwords
nltk.download('stopwords')
nltk.download('punkt')
from nltk.tokenize import word_tokenize
stop_words = set(stopwords.words("english"))

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\fishrak\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\fishrak\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [4]:
def all_file_paths(master_directory):   
    business_txt_list = []
    path_l = []
    for root, dirs, files in os.walk(master_directory):
        
        

        if files:
            for i in files:
                path = "{}\{}".format(root,i)
                if path == 'bbc_doc\desktop.ini':
                    continue
                path_l.append(path)
                
    
    return(path_l)

all_path = all_file_paths('bbc_doc')

for i in all_path:
    print(i)

bbc_doc\business\001.docx
bbc_doc\business\002.docx
bbc_doc\business\003.docx
bbc_doc\business\004.docx
bbc_doc\business\005.docx
bbc_doc\entertainment\001.docx
bbc_doc\entertainment\002.docx
bbc_doc\entertainment\003.docx
bbc_doc\entertainment\004.docx
bbc_doc\entertainment\005.docx
bbc_doc\politics\001.docx
bbc_doc\politics\002.docx
bbc_doc\politics\003.docx
bbc_doc\politics\004.docx
bbc_doc\politics\005.docx
bbc_doc\sports\001.docx
bbc_doc\sports\002.docx
bbc_doc\sports\003.docx
bbc_doc\sports\004.docx
bbc_doc\sports\005.docx
bbc_doc\technology\001.docx
bbc_doc\technology\002.docx
bbc_doc\technology\003.docx
bbc_doc\technology\004.docx
bbc_doc\technology\005.docx


In [5]:

def preprocessor(text):
    text = re.sub('<[^>]*>', '', text)
    text = re.sub('[\W]+', '', text.lower())
    return text


def findline(word,doc_list): #Added doc_list as var
    line_num = []
    for i in range(len(doc_list)):
        if word in doc_list[i]:
            line_num.append(i+1)

    return line_num



In [68]:
def sim_word(path,query,score=0.6):
 
#FI
    text = docx2txt.process(path)


    tokens = nlp(text)
    lemma_list = []
    for token in tokens:
        if token.is_stop is False:
            token_preprocessed = preprocessor(token.text)
            if token_preprocessed != '':
                 lemma_list.append(nlp(token_preprocessed))
   

#     tokens_str = text.lower()
#     tokens_str = word_tokenize(tokens_str)
#     tokens_str = [i for i in tokens_str if not i in stop_words]  
    

    
    l_q = len(nlp(query))
    
    if l_q >= 2:
        
        bi_g = []
        c=0
        while c < len(lemma_list)-1:

            bi_g.append(nlp(lemma_list[c].text+" "+lemma_list[c+1].text))
            c = c+1


        lemma_list = lemma_list+bi_g
    

    
    
    
#PN    
    wordfreq = []
    for w in lemma_list:
        wordfreq.append(lemma_list.count(w))


    for i in list(zip(lemma_list, wordfreq)):
        if query == i[0]:
            word_occurences = (i[0], i[1])
            break

    line = 0
    word_not_found = []

    for word in text:
        if word == '\n':
            line += 1

    doc_list = text.lower().splitlines()
    doc_list = [i for i in doc_list if i]

#PN


#FI & PN
    key = nlp(query)
    
    sim_score = []
    words = []

    pg_list = []

    for i in lemma_list:
        s = key.similarity(i)

        if s > score:

            word_found_list = []
            word_found_list.append(str(i.text))
            word_found_lines = findline(word_found_list[0],doc_list)
            
            if word_found_lines:
                pass
            else:
                continue


            pg_list.append(tuple(word_found_lines))
            words.append(i)
            sim_score.append(s)
            

    
            
    matrix = pd.DataFrame({'Similarity Score': sim_score,'paragraph': pg_list}, index = words)
    matrix = matrix.drop_duplicates()

    matrix =  matrix.sort_values(by=['Similarity Score'],ascending=False)

    print("For document: {}, path: {}, keyword: {}, Similarity Score Threshold: {}".format(os.path.basename(path),path,query,score))
    print(matrix)
    print('\n')

#FI & PN

In [72]:
for i in all_path[:1]:
    sim_word(i,"profit margin",0.6)



For document: 001.docx, path: bbc_doc\business\001.docx, keyword: profit margin, Similarity Score Threshold: 0.6
                         Similarity Score        paragraph
(profit, margins)                0.939976             (5,)
(profit)                         0.856083  (1, 2, 3, 4, 5)
(margins)                        0.801575             (5,)
(quarter, profits)               0.796366           (4, 5)
(wider, profit)                  0.775118             (5,)
(profits)                        0.772505     (2, 3, 4, 5)
(underlying, profit)             0.756716             (4,)
(revenue)                        0.739660        (4, 5, 6)
(higher, revenue)                0.737573             (5,)
(quarterly, profits)             0.734836             (2,)
(profit, dip)                    0.731868             (3,)
(earnings, growth)               0.725828             (5,)
(operating, earnings)            0.712284             (5,)
(earnings)                       0.695853             (5,)
(p

In [55]:
x = set()

if x:
    print('T')
else:
    print('F')

F


In [31]:
l = ["at","at","at","at"]
l

['at', 'at', 'at', 'at']

In [39]:
l2 = []
i=0
while i < len(l)-1:
    print(i)
    l2.append(l[i]+" "+l[i+1])
    i = i+1
l + l2

0
1
2


['at', 'at', 'at', 'at', 'at at', 'at at', 'at at']

### Trying with transformer models 

In [8]:
# from sentence_transformers import SentenceTransformer, util
# import numpy as np

# model = SentenceTransformer('stsb-roberta-large')

# sentence1 = "Word"
# sentence2 = "Word"
# # encode sentences to get their embeddings
# embedding1 = model.encode(sentence1, convert_to_tensor=True)
# embedding2 = model.encode(sentence2, convert_to_tensor=True)
# # compute similarity scores of two embeddings
# cosine_scores = util.pytorch_cos_sim(embedding1, embedding2)
# print("Sentence 1:", sentence1)
# print("Sentence 2:", sentence2)
# print("Similarity score:", cosine_scores.item())


# key = "learn"

# embedding1 = model.encode(key, convert_to_tensor=True)

# for i in lemma_list:
#     i = i.text
     
#     embedding2 = model.encode(i, convert_to_tensor=True)
#     # compute similarity scores of two embeddings
#     cosine_scores = util.pytorch_cos_sim(embedding1, embedding2)
    
#     if cosine_scores > 0.5:
#         print("Key:{} \nWord Found:{} \nSimilarity score:{:0.2f}".format(key,i,np.array(cosine_scores)[0][0]))
#         print("----------------------------------------------------")  